In [1]:
import pandas as pd

# File paths
file1_path = "customers_data.csv"
file2_path = "sales_data-2.csv"

# Load CSV files
df1 = pd.read_csv(file1_path)
df2 = pd.read_csv(file2_path)

# Convert 'Customer_ID' from hexadecimal to decimal (numeric)
df1['Customer_ID'] = df1['Customer_ID'].apply(lambda x: int(x, 16))
df2['Customer_ID'] = df2['Customer_ID'].apply(lambda x: int(x, 16))

# Define new file names
new_file1_path = "customers_data_new.csv"
new_file2_path = "sales_data-2_new.csv"

# Save modified dataframes to new CSV files
df1.to_csv(new_file1_path, index=False)
df2.to_csv(new_file2_path, index=False)

# Return paths of new files
new_file1_path, new_file2_path


('customers_data_new.csv', 'sales_data-2_new.csv')

In [3]:
import pandas as pd

# Load sales data
sales_file_path = "sales_data-2.csv"  # Change this to your actual file path
df_sales = pd.read_csv(sales_file_path)

# Create Total_Amt column
# Total Amount = Quantity * Unit Price
df_sales["Total_Amt"] = df_sales["Quantity"] * df_sales["Unit_Price"]

# Group by Customer_ID and sum Total_Amt
df_grouped = df_sales.groupby("Customer_ID", as_index=False)["Total_Amt"].sum()

# Merge back with original data
df_sales = df_sales.merge(df_grouped, on="Customer_ID", suffixes=("", "_Total"))

# Save to new CSV file
new_sales_file_path = "sales_data-2_new.csv"
df_sales.to_csv(new_sales_file_path, index=False)

print(f"Processed sales data saved to: {new_sales_file_path}")

Processed sales data saved to: sales_data-2_new.csv
